# Лабораторная работа 4. ООП.

In [1]:
import pylab
import numpy as np
import matplotlib.pyplot as plt

# 1. Создание классов и объектов
В языке программирования Python классы создаются с помощью инструкции class, за которой следует произвольное имя класса, после которого ставится двоеточие, далее с новой строки и с отступом реализуется тело класса:

In [2]:
class A:    #  class ИмяКласса:
    pass    #  тело_класса

Создание экземпляра класса:

In [3]:
a = A()     #  имя_переменной = ИмяКласса()

In [4]:
print(a, 'объект класса', type(a))

<__main__.A object at 0x00000178AB41DBE0> объект класса <class '__main__.A'>


# 1.1. Класс как модуль (библиотека)
Класс можно представить подобно модулю (библиотеки), в нем могут быть свои переменные со значениями и функции, у класса есть собственное пространство имен, доступ к которым возможен через имя класса:

In [5]:
class CLASS:
    const = 5              # атрибут класса
    def adder(v):          # функция-метод
        return v + CLASS.const

In [6]:
CLASS.const

5

In [7]:
CLASS.adder(4)

9

# 1.2. Класс как создатель объектов


In [8]:
Object = CLASS()

In [9]:
Object.const

5

In [10]:
Object.adder(100)

TypeError: adder() takes 1 positional argument but 2 were given

Дело в том, что классы и объекты не просто модули. Класс создает объекты, которые в определенном смысле являются его наследниками (копиями). 

Это значит, что если у объекта нет собственного поля const, то интерпретатор ищет его уровнем выше, то есть в классе. Таким образом, если мы присваиваем объекту поле с таким же именем как в классе, то оно перекрывает, т. е. переопределяет, поле класса:

In [11]:
Object.const

5

In [12]:
Object.const = 10
Object.const

10

In [13]:
CLASS.const

5

Видно что Object.const и CLASS.const – это разные переменные. 

Object.const находится в пространстве имен объекта Object. 

CLASS.const – в пространстве класса CLASS. 

Если не задовать поле const объекту Object, то интерпретатор бы поднялся выше по дереву наследования и пришел бы в класс, где бы и нашел это поле.

Методы также наследуются объектами класса. В данном случае у объекта Object нет своего собственного метода adder, значит, он ищется в классе CLASS. Однако, от класса B может быть порождено множество объектов. И методы предназначаются для обработки объектов. Таким образом, когда вызывается метод, в него надо передать конкретный объект, который он будет обрабатывать.


Выражение Object.adder(100) выполняется интерпретатором следующим образом:

- Ищу атрибут adder() у объекта Object. Не нахожу.

- Тогда иду искать в класс CLASS, так как он создал объект Object.

- Здесь нахожу искомый метод. Передаю ему объект, к которому этот метод надо применить, и аргумент, указанный в скобках.

Другими словами, выражение Object.adder(100) преобразуется в выражение CLASS.adder(Object, 100).

Таким образом, интерпретатор попытался передать в метод adder() класса CLASS два параметра – объект Object и число 100. Но мы запрограммировали метод adder() так, что он принимает только один параметр. 

Однако:

In [14]:
Object.adder()

TypeError: unsupported operand type(s) for +: 'CLASS' and 'int'

Получается странная ситуация. Ведь adder() вызывается не только через класса, но и через порожденные от него объекты. Однако в последнем случае всегда будет возникать ошибка. 

Может понадобиться метод с параметрами, но которому не надо передавать экземпляр данного класса. Для таких ситуаций предназначены статические методы. Такие методы могут вызываться через объекты данного класса, но сам объект в качестве аргумента в них не передается.

В Python острой необходимости в статических методах нет, так как код может находиться за пределами класса, и программа не начинает выполняться из класса. Если нам нужна просто какая-нибудь функция, мы можем определить ее в основной ветке. Однако в Python тоже можно реализовать подобное, то есть статические методы, с помощью декоратора @staticmethod:

In [15]:
class CLASS:
    const = 5              # атрибут класса
    @staticmethod    
    def adder(v):          # функция-метод
        return v + CLASS.const

In [16]:
Object = CLASS()

In [17]:
Object.adder(5)

10

Статические методы в Python – по-сути обычные функции, помещенные в класс для удобства и находящиеся в пространстве имен этого класса. Это может быть какой-то вспомогательный код. 

Вообще, если в теле метода не используется self, то есть ссылка на конкретный объект, имеет смысл сделать метод статическим.

# 1.3. Изменение полей объекта

В Python объекту можно не только переопределять поля и методы, унаследованные от класса, также можно добавить новые, которых нет в классе:

In [18]:
Object1 = CLASS()
Object2 = CLASS()

In [19]:
Object2.str = 'abcd'
Object2.str

'abcd'

In [20]:
Object1.str

AttributeError: 'CLASS' object has no attribute 'str'

In [21]:
CLASS.str

AttributeError: type object 'CLASS' has no attribute 'str'

Однако в программировании так делать не принято, потому что тогда объекты одного класса будут отличаться между собой по набору атрибутов. Это затруднит автоматизацию их обработки, внесет в программу хаос.

Поэтому принято присваивать полям, а также получать их значения, путем вызова методов (сеттеров (set – установить) и геттеров (get – получить)):

In [22]:
class CLASS:
    def setName(self, n):
        self.name = n 
    def getName(self):
        try:
            return self.name
        except:
            return "No name"

In [23]:
first = CLASS()
second = CLASS()

In [24]:
first.setName("Bob")
first.getName()

'Bob'

In [25]:
print(second.getName())

No name


# 1.4. Специальные методы

# Конструктор класса (метод $__init__()$)

В объектно-ориентированном программировании конструктором класса называют метод, который автоматически вызывается при создании объектов. Его также можно назвать конструктором объектов класса. Имя такого метода обычно регламентируется синтаксисом конкретного языка программирования. 

В Python роль конструктора играет метод $__init__()$.

В Python наличие пар знаков подчеркивания спереди и сзади в имени метода говорит о том, что он принадлежит к группе методов перегрузки операторов. Если подобные методы определены в классе, то объекты могут участвовать в таких операциях как сложение, вычитание, вызываться как функции и др.

При этом методы перегрузки операторов не надо вызывать по имени. Вызовом для них является сам факт участия объекта в определенной операции. В случае конструктора класса – это операция создания объекта. Так как объект создается в момент вызова класса по имени, то в этот момент вызывается метод $__init__()$, если он определен в классе.

Необходимость конструкторов связана с тем, что нередко объекты должны иметь собственные свойства сразу. 

Пусть имеется класс Person, объекты которого обязательно должны иметь имя и фамилию:

In [26]:
class Person:
    def setName(self, n, s):
        self.name = n
        self.surname = s

In [27]:
p1 = Person()
p1.setName("Bill", "Ross")

ИЛИ

In [28]:
class Person:
    def __init__(self, n, s):
        self.name = n
        self.surname = s

В свою очередь, конструктор класса не позволит создать объект без обязательных полей:

In [29]:
p2 = Person()

TypeError: __init__() missing 2 required positional arguments: 'n' and 's'

In [30]:
p2 = Person("Sam", "Baker")
print(p2.name, p2.surname)

Sam Baker


Однако бывает, что надо допустить создание объекта, даже если никакие данные в конструктор не передаются. В таком случае параметрам конструктора класса задаются значения по умолчанию:

In [31]:
class Rectangle:
    def __init__(self, w = 0.5, h = 1):
        self.width = w
        self.height = h
    def square(self):
        return self.width * self.height

In [32]:
rec1 = Rectangle(5, 2)
rec2 = Rectangle()
rec3 = Rectangle(3)
rec4 = Rectangle(h = 4)

print(rec1.square())
print(rec2.square())
print(rec3.square())
print(rec4.square())

10
0.5
3
2.0


# Конструктор и деструктор

Помимо конструктора объектов в языках программирования есть обратный ему метод – деструктор. Он вызывается для уничтожения объекта.

В языке программирования Python объект уничтожается, когда исчезают все связанные с ним переменные или им присваивается другое значение, в результате чего связь со старым объектом теряется. Удалить переменную можно с помощью команды языка del.

В классах Python функцию деструктора выполняет метод $__del__()$.

In [33]:
class Student:
 
    def __init__(self, name, surname, position=1):
        self.name = name
        self.surname = surname
        self.position = position
 
    def display(self):
        return self.name, self.surname, self.position
 
    def __del__(self):
        print ("Goodbye %s %s" %(self.name, self.surname))

In [34]:
p1 = Student('big', 'dude', 3) 
p2 = Student('small', 'croon', 4)
p3 = Student('neutral', 'guy', 5)

In [35]:
print (p1.display())
print (p2.display())
print (p3.display())

('big', 'dude', 3)
('small', 'croon', 4)
('neutral', 'guy', 5)


In [36]:
del p2

Goodbye small croon


In [37]:
print (p2.display())

NameError: name 'p2' is not defined

# Специальные методы Python

В Python есть ряд зарезервированных имен методов создаваемого класса - специальные (или стандартные) методы.

Более подробную информацию о них вы можете найти в соответствующей документации по Python: https://docs.python.org/3/reference/datamodel.html

Например:

$__bool__()$

Возвращает True или False.

$__call__()$

Позволяет использовать объект как функцию, т.е. его можно вызвать.

$__len__()$

Чаще всего реализуется в коллекциях и сходными с ними по логике работы типами, которые позволяют хранить наборы данных. Для списка (list) __len__() возвращает количество элементов в списке, для строки – количество символов в строке. Вызывается функцией len(), встроенной в язык Python.

# Метод $__setattr__()$

В Python атрибуты объекту можно назначать за пределами класса:

In [38]:
class A:
    def __init__(self, v):
        self.field1 = v

In [39]:
a = A(10)
a.field2 = 20
print(a.field1, a.field2)

10 20


Если такое поведение нежелательно, его можно запретить с помощью метода перегрузки оператора присваивания атрибуту $__setattr__()$:

In [40]:
class A:
    def __init__(self, v):
        self.field1 = v
    def __setattr__(self, attr, value):
        if attr == 'field1':
            self.__dict__[attr] = value
        else:
            raise AttributeError('Произошло обращение к несуществующему отребуту!')

In [41]:
a = A(15)
a.field1

15

In [42]:
a.field2 = 30

AttributeError: Произошло обращение к несуществующему отребуту!

In [43]:
a.field2

AttributeError: 'A' object has no attribute 'field2'

In [44]:
a.__dict__

{'field1': 15}

Метод __setattr__(), если он присутствует в классе, вызывается всегда, когда какому-либо атрибуту выполняется присваивание. Обратите внимание, что присвоение несуществующему атрибуту также обозначает его добавление к объекту.

Когда создается объект a, в конструктор передается число 15. Здесь для объекта заводится атрибут field1. Факт попытки присвоения ему значения тут же отправляет интерпретатор в метод __setattr__(), где проверяется соответствует ли имя атрибута строке 'field1'. Если так, то атрибут и соответствующее ему значение добавляется в словарь атрибутов объекта.

Нельзя в __setattr__() написать просто self.field1 = value, так как это приведет к новому рекурсивному вызову метода __setattr__(). Поэтому поле назначается через словарь __dict__, который есть у всех объектов, и в котором хранятся их атрибуты со значениями.

Если параметр attr не соответствует допустимым полям, то искусственно возбуждается исключение AttributeError. Мы это видим, когда в основной ветке пытаемся обзавестись полем field2.

# Пример №1. Числа Фибоначчи

Последовательность чисел Фибоначчи задаётся рекуррентным выражением:

$$ F_n =  \begin{cases}
           0, n = 0, \\
           1, n = 1, \\
           F_{n-1}+F_{n-2}, n > 1.
          \end{cases} $$

Что даёт следующую последовательность {0, 1, 1, 2, 3, 5, 8, 13, 21, 34, …}.

Один из способов решения, который может показаться логичным и эффективным, — решение с помощью рекурсии:

In [45]:
def Fibonacci_Recursion(n):
    if n == 0:
        return 0
    if n == 1:
        return 1
    return Fibonacci_Recursion (n-1) + Fibonacci_Recursion (n-2)

Используя такую функцию, мы будем решать задачу «с конца» — будем шаг за шагом уменьшать n, пока не дойдем до известных значений.

Но, как мы видели ранее эта реализация многократно повторяет решение одних и тех же задач. Это связано с тем, что одни и те же промежуточные данные вычисляются по несколько раз, а число операций нарастает с той же скоростью, с какой растут числа Фибоначчи — экспоненциально.

Один из выходов из данной ситуации — сохранение уже найденных промежуточных результатов с целью их повторного использования (кеширование). Причём кеш должен храниться во внешней области памяти.

In [46]:
def Fibonacci_Recursion_cache(n, cache):
    if n == 0:
        return 0
    if n == 1:
        return 1
    if cache[n] > 0:
        return cache[n]
    cache[n] = Fibonacci_Recursion_cache (n-1, cache) + Fibonacci_Recursion_cache (n-2, cache)
    return cache[n]

Приведенное решение достаточно эффективно (за исключением накладных расходов на вызов функций). Но можно поступить ещё проще:

In [47]:
def Fibonacci(n):
    
    fib = [0]*max(2,n)
    fib[0] = 1
    fib[1] = 1
    for i in range(2, n):
        fib[i] = fib[i - 1] + fib[i - 2]
        
    return fib[n-1]

Такое решение можно назвать решением «с начала» — мы первым делом заполняем известные значения, затем находим первое неизвестное значение, потом следующее и т.д., пока не дойдем до нужного.

Так и работает динамическое программирование: сначала решили все подзадачи (нашли все F[i] для i < n), затем, зная решения подзадач, нашли решение исходной задачи.

# Упражнение №1

Создайте класс для вычисления чисел Фибоначчи. Каждое число фибоначи является объектом этого класса, которое имеет атрибуты: значение и номер. Используйте функции для инициализации (вычисления) чисел Фибоначчи как сторонние по отношению к этому классу.

# Упражнение №2

Поместите функции для вычисления чисел Фибоначчи внутрь созданного класса как статические функции.

# Упражнение №3

Перегрузите опарации сложения, вычитания, умножения и деления для созданного класса как операции с номерами чисел Фибоначи.

# 1.5. Инкапсуляция
Под инкапсуляцией в объектно-ориентированном программировании понимается упаковка данных и методов для их обработки вместе, т. е. в классе. В Python инкапсуляция реализуется как на уровне классов, так и объектов. В ряде других языков, например в Java, под инкапсуляцией также понимают сокрытие свойств и методов, в результате чего они становятся приватными. Это значит, что доступ к ним ограничен либо пределами класса, либо модуля.

В Python подобной инкапсуляции нет, хотя существует способ ее имитировать. Перед тем как выяснять, как это делается, надо понять, зачем вообще что-то скрывать.

Дело в том, что классы бывают большими и сложными. В них может быть множество вспомогательных полей и методов, которые не должны использоваться за его пределами. Они просто для этого не предназначены. Они своего рода внутренние шестеренки, обеспечивающие нормальную работу класса.

Кроме того, в других языках программирования хорошей практикой считается сокрытие всех полей объектов, чтобы уберечь их от прямого присвоения значений из основной ветки программы. Их значения можно изменять и получать только через вызовы методов, специально определенных для этих целей.

Например, если надо проверять присваиваемое полю значение на корректность, то делать это каждый раз в основном коде программы будет неправильным. Проверочный код должен быть помещен в метод, который получает данные, для присвоения полю. А само поле должно быть закрыто для доступа из вне класса. В этом случае ему невозможно будет присвоить недопустимое значение.

Часто намеренно скрываются поля самого класса, а не его объектов. Например, если класс имеет счетчик своих объектов, то необходимо исключить возможность его случайного изменения из вне. Рассмотрим пример с таким счетчиком на языке Python.

In [1]:
class B:
    count = 0
    def __init__(self):
        B.count += 1
    def __del__(self):
        B.count -= 1

In [2]:
a = B()
b = B()
с = B()
print(b.count)
print(B.count) 
del a
print(B.count) 

3
3
2


In [4]:
b.count

2

In [5]:
B.count -= 1
print(B.count)

1


# 2. Полиморфизм
Полиморфизм в объектно-ориентированном программировании – это возможность обработки разных типов данных, т. е. принадлежащих к разным классам, с помощью "одно и той же" функции, или метода. На самом деле одинаковым является только имя метода, его исходный код зависит от класса. Кроме того, результаты работы одноименных методов могут существенно различаться. Поэтому в данном контексте под полиморфизмом понимается множество форм одного и того же слова – имени метода.

Например, два разных класса содержат метод total, однако инструкции каждого предусматривают совершенно разные операции. Так в классе T1 – это прибавление 10 к аргументу, в T2 – подсчет длины строки символов. В зависимости от того, к объекту какого класса применяется метод total, выполняются те или иные инструкции.

In [8]:
class T1:
     n=10
     def total(self, N):
          self.total = int(self.n) + int(N)
 
class T2:
     def total(self,s):
          self.total = len(str(s))

In [9]:
t1 = T1()
t2 = T2()
t1.total(45)
t2.total(45)
print(t1.total) 
print(t2.total) 

55
2


Как видно полиморфизмом обладают классы связанные наследованием. У каждого может быть свой метод __init__() или square() или какой-нибудь другой. Какой именно из методов square() вызывается, и что он делает, зависит от принадлежности объекта к тому или иному классу.

Однако классы не обязательно должны быть связанны наследованием. Полиморфизм как один из ключевых элементов ООП существует независимо от наследования. Классы могут быть не родственными, но иметь одинаковые методы.

Полиморфизм дает возможность реализовывать так называемые единые интерфейсы для объектов различных классов. Например, разные классы могут предусматривать различный способ вывода той или иной информации объектов. Однако одинаковое название метода вывода позволит не запутать программу, сделать код более ясным.

Рассмотрим пример полиморфизма на еще одном методе, который перегружает функцию print().

Если вы создадите объект собственного класса, а потом попробуете вывести его на экран, то получите информацию о классе объекта и его адрес в памяти. Такое поведение функции print() по-умолчанию по отношению к пользовательским классам запрограммировано на самом верхнем уровне иерархии, где-то в суперклассе, от которого неявно наследуются все остальные.

In [10]:
class A:
    def __init__(self, v1, v2):
        self.field1 = v1
        self.field2 = v2

In [11]:
a = A(3, 4)
print(a)

Если же мы хотим, чтобы, когда объект передается функции print(), выводилась какая-нибудь другая более полезная информация, то в класс надо добавить специальный метод __str__(). Этот метод должен обязательно возвращать строку, которую будет выводить функция print():

In [12]:
class A:
    def __init__(self, v1, v2):
        self.field1 = v1
        self.field2 = v2
    def __str__(self):
        return str(self.field1) + " " + str(self.field2)

In [13]:
a = A(3, 4)
print(a)

3 4


# 3. Наследование
Наследование – важная составляющая объектно-ориентированного программирования. Так или иначе мы уже сталкивались с ним, ведь объекты наследуют атрибуты своих классов. Однако обычно под наследованием в ООП понимается наличие классов и подклассов. Также их называют супер- или надклассами и классами, а также родительскими и дочерними классами.

Суть наследования здесь схожа с наследованием объектами от классов. Дочерние классы наследуют атрибуты родительских, а также могут переопредять атрибуты и добавлять свои.

# 3.1. Простое наследование

In [14]:
class Table:
    def __init__(self, l, w, h):
        self.lenght = l
        self.width = w
        self.height = h
 
class KitchenTable(Table):
    def setPlaces(self, p):
        self.places = p
 
class DeskTable(Table):
    def square(self):
        return self.width * self.lenght

В данном случае классы KitchenTable и DeskTable не имеют своих собственных конструкторов, поэтому наследуют его от родительского класса. При создании экземпляров этих столов, передавать аргументы для __init__() обязательно, иначе возникнет ошибка:

In [15]:
t1 = KitchenTable()

TypeError: __init__() missing 3 required positional arguments: 'l', 'w', and 'h'

In [16]:
t1 = KitchenTable(2, 2, 0.7)
t2 = DeskTable(1.5, 0.8, 0.75)
t3 = KitchenTable(1, 1.2, 0.8)

In [17]:
t3.lenght

1

Несомненно можно создавать столы и от родительского класса Table. Однако он не будет, согласно неким родственным связям, иметь доступ к методам setPlaces() и square(). Точно также как объект класса KitchenTable не имеет доступа к единоличным атрибутам сестринского класса DeskTable

In [18]:
t4 = Table(1, 1, 0.5)

In [19]:
t2.width * t2.lenght

1.2000000000000002

In [20]:
t2.square()

1.2000000000000002

In [21]:
t4.square()

AttributeError: 'Table' object has no attribute 'square'

In [22]:
t3.square()

AttributeError: 'KitchenTable' object has no attribute 'square'

# 3.2. Полное переопределение метода надкласса
Что если в подклассе нам не подходит код метода его надкласса. Допустим, мы вводим еще один класс столов, который является дочерним по отношению к DeskTable. Пусть это будут компьютерные столы, при вычислении рабочей поверхности которых надо отнимать заданную величину. Имеет смысл внести в этот новый подкласс его собственный метод square():

In [23]:
class ComputerTable(DeskTable):
    def square(self, e):
        return self.width * self.lenght - e

При создании объекта типа ComputerTable по-прежнему требуется указывать параметры, так как интерпретатор в поисках конструктора пойдет по дереву наследования сначала в родителя, а потом в прародителя и найдет там метод __init__().

Однако когда будет вызываться метод square(), то поскольку он будет обнаружен в самом ComputerTable, то метод square() из DeskTable останется невидимым, т. е. для объектов класса ComputerTable он окажется переопределенным.

In [24]:
ct = ComputerTable(2, 1, 1)
ct.square(0.3)

1.7

ИЛИ

In [25]:
class ComputerTable(DeskTable):
    def square(self, e):
        return DeskTable.square(self) - e 

In [26]:
ct = ComputerTable(2, 1, 1)
ct.square(0.3)

1.7

Допустим, в классе KitchenTable нам не нужен метод, поле places должно устанавливаться при создании объекта в конструкторе. В классе можно создать собсвенный конструктор с чистого листа, чем переопределить родительский:

In [27]:
class KitchenTable(Table):
    def __init__(self, l, w, h, p):
        self.length = l
        self.width = w
        self.height = h
        self.places = p

Однако, если дублируется почти весь конструктор надкласса, проще вызвать родительский конструктор, после чего дополнить своим кодом:

In [28]:
class KitchenTable(Table):
    def __init__(self, l, w, h, p):
        Table.__init__(self, l, w, h)
        self.places = p

In [29]:
tk = KitchenTable(2, 1.5, 0.7, 10)

In [30]:
tk.places

10

In [31]:
tk.width 

1.5

# 3.3. Множественное наследование

Для наследования от нескольких классав в определении класса-наследника достаточно указать в круглых скобках сразу несколько базовых классов через запятую.

# Пример.

Рассмотрим три класса:

In [6]:
class Classl: 
    def funс1(self):
        print("Метод funс1() класса Classl")

class Class2(Classl): # Простое наследование
    def func2(self):
        print("Метод func2() класса Class2")

class Class3(Classl): # Простое наследование
    def funс1(self):
        print("Метод funс1() класса Class3")
    def func2(self):
        print("Метод func2() класса Class3")
    def func3(self):
        print("Метод func3() класса Class3")
    def func4(self):
        print("Метод func4() класса Class3")

class Class4(Class2, Class3): # Множественное наследование
    def func4(self):
        print("Метод func4() класса Class4")

In [8]:
c = Class4()
c.funс1() 
c.func2() 
c.func3() 
c.func4() 

Метод funс1() класса Class3
Метод func2() класса Class2
Метод func3() класса Class3
Метод func4() класса Class4


Метод func1() определен в двух классах: class1 и ciass3. Так как вначале просматриваются все базовые классы, непосредственно указанные в определении текущего класса, метод func1() будет найден в классе class3 (поскольку он указан в числе базовых классов в определении Class4), а не в классе Class1.

Метод func2() также определен в двух классах: Class2 и Class3. Так как класс Class2 стоит первым в списке базовых классов, то метод будет найден именно в нем. 

Чтобы наследовать метод из класса Class3, следует указать это явным образом:

In [9]:
class Class4(Class2, Class3): # Множественное наследование
# Наследуем func2() из класса Class3, а не из класса Class2
    func2 = Class3.func2
    def func4(self):
        print("Метод func4() класса Class4")

In [10]:
c = Class4()
c.funс1() 
c.func2() 
c.func3() 
c.func4() 

Метод funс1() класса Class3
Метод func2() класса Class3
Метод func3() класса Class3
Метод func4() класса Class4


# 4. Композиция

Еще одной особенностью объектно-ориентированного программирования является возможность реализовывать так называемый композиционный подход. Заключается он в том, что есть класс-контейнер, он же агрегатор, который включает в себя вызовы других классов. В результате чего, что при создании объекта класса-контейнера, также создаются объекты включенных в него классов.

Не следует путать композицию с наследованием, в том числе множественным. Наследование предполагает принадлежность к какой-то общности (похожесть), а композиция — формирование целого из частей. Наследуются атрибуты, т.е. возможности, другого класса, при этом объектов непосредственно родительского класса не создается. При композиции же класс-агрегатор создает объекты других классов.

Рассмотрим на примере реализацию композиции в Python. Пусть, требуется написать программу, которая вычисляет площадь стен. При этом окна, двери, пол и потолок должны быть исключены.

И так, комната – это прямоугольный параллелепипед, состоящий из шести прямоугольников. Его площадь представляет собой сумму площадей составляющих его прямоугольников. Площадь прямоугольника равна произведению его длины на ширину.

In [32]:
class WinDoor:
     def __init__(self, x, y):
          self.square = x * y 

In [33]:
class Room:
    def __init__(self, x, y, z):
        self.square = 2 * z * (x + y)
        self.wd = []
    def addWD(self, w, h):
        self.wd.append(WinDoor(w, h))
    def workSurface(self):
        new_square = self.square
        for i in self.wd:
            new_square -= i.square
        return new_square

In [34]:
r1 = Room(6, 3, 2.7) 
print(r1.square)
r1.addWD(1, 1) 
r1.addWD(1, 1)
r1.addWD(1, 2)
print(r1.workSurface())

48.6
44.6


# Домашнее задание (базовое):

# Задание №1. 

Создать класс с двумя переменными. Добавить функцию вывода на экран и функцию изменения этих переменных. Добавить функцию, которая находит сумму значений этих переменных, и функцию которая находит наибольшее значение из этих двух переменных.

# Задание №2. 

Составить описание класса многочленов от одной переменной, задаваемых степенью многочлена и массивом коэффициентов. Предусмотреть методы для вычисления значения многочлена для заданного аргумента, операции сложения, вычитания и умножения многочленов с получением нового объекта-многочлена, вывод на экран описания многочлена.

# Задание №3

Составить описание класса для вектора, заданного координатами его концов в трехмерном пространстве. Обеспечить операции сложения и вычитания векторов с получением нового вектора (суммы или разности), вычисления скалярного произведения двух векторов, длины вектора, косинуса угла между векторами.

# Домашнее задание (дополнительное):

# Поезда.

Создайте структуру с именем train, содержащую поля: название пунктов отправления и назначения, время отправления и прибытия. Перегрузить операцию сложения - два поезда можно сложить, если пунк назначения первого совпадает с пунктом отправления второго и время прибытия первого раньше чем отправление второго.

# Обобщённое число.

Создайте класс обобщающий понятие комплексных, двойных и дуальных чисел. И на его основе создайте классы этих чисел.

Такие числа объеденены одной формой записи:

$$ c = a + ib $$

где c - обобщённое число (комплексное, двойное или дуальное), a и b - вещественные числа, i - некоммутирующий символ.

Именно из-за наличия символа i число c не просто сумма a и b. Такие числа можно представлять как вектор на плоскости (a,b).

А символ i обладает следующим свойством:

- для комплексных чисел

$$ i^2 = -1 $$

- для двойных чисел

$$ i^2 = 1 $$

- для дуальных чисел

$$ i^2 = 0 $$

Перегрузить для них базовые операции: сложения, вычитания, умножения и деления.

Например, операция умножения для таких чисел имеет вид:

$$ (a_1+b_1i)\cdot (a_2+b_2i)=a_1a_2+b_1a_2i+a_1b_2i+b_1b_2i^{2}=(a_1a_2+b_1b_2i^{2})+(b_1a_2+a_1b_2)i. $$